# Establish benchmark performance

Add parent directory to `sys.path` so that we can import `ae483tools`.

In [ ]:
import sys, os
sys.path.append(os.path.abspath('..'))

Do all other imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ae483tools import *

Load and resample data.

In [ ]:
# Load data
raw_data_drone, raw_data_mocap, raw_data_bodies = load_hardware_data(
    'hardware_data_requirements.json',
)

# Resample data
data_drone = resample_data_drone(
    raw_data_drone,
    t_min_offset=0.,
    t_max_offset=0.,
)

# Get only the subset of data that corresponds to 10 seconds at hover
only_in_flight(data_drone, t_interval=10.)

Parse data.

In [ ]:
# time
t = data_drone['time']

# position
p_x = data_drone['stateEstimate.x']
p_y = data_drone['stateEstimate.y']
p_z = data_drone['stateEstimate.z']

# desired position
p_x_des = data_drone['ctrltarget.x']
p_y_des = data_drone['ctrltarget.y']
p_z_des = data_drone['ctrltarget.z']

# orientation
psi = np.deg2rad(data_drone['stateEstimate.yaw']) 
theta = - np.deg2rad(data_drone['stateEstimate.pitch'])
phi = np.deg2rad(data_drone['stateEstimate.roll'])

Plot position, desired position, and orientation.

In [ ]:
fig, (ax_pos, ax_ori) = plt.subplots(2, 1, figsize=(8, 4), sharex=True, tight_layout=True)
px = ax_pos.plot(t, p_x, label=f'p_x')
py = ax_pos.plot(t, p_y, label=f'p_y')
pz = ax_pos.plot(t, p_z, label=f'p_z')
ax_pos.plot(t, p_x_des, '--', label=f'p_x (desired)', color=px[0].get_color())
ax_pos.plot(t, p_y_des, '--', label=f'p_y (desired)', color=py[0].get_color())
ax_pos.plot(t, p_z_des, '--', label=f'p_z (desired)', color=pz[0].get_color())
ax_pos.legend()
ax_pos.grid()
ax_ori.plot(t, psi, label='psi')
ax_ori.plot(t, theta, label='theta')
ax_ori.plot(t, phi, label='phi')
ax_ori.legend()
ax_ori.grid()
ax_ori.set_xlabel('time (s)')
plt.show()

Compute RMSE in each component of position and orientation.

In [ ]:
p_x_rmse = np.sqrt(np.mean((p_x - p_x_des)**2))
p_y_rmse = np.sqrt(np.mean((p_y - p_y_des)**2))
p_z_rmse = np.sqrt(np.mean((p_z - p_z_des)**2))
psi_rmse = np.sqrt(np.mean((psi - 0.)**2))
theta_rmse = np.sqrt(np.mean((theta - 0.)**2))
phi_rmse = np.sqrt(np.mean((phi - 0.)**2))

Generate the text for the first row of results in a table of RMSE values.

In [ ]:
print(f'| BENCHMARK | {p_x_rmse:.3f} | {p_y_rmse:.3f} | {p_z_rmse:.3f} | {psi_rmse:.3f} | {theta_rmse:.3f} | {phi_rmse:.3f} | DEFAULT CONTROLLER |')